In [1]:
### MODEL TRAINING ###

In [2]:
# import modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.pipeline import make_pipeline
from lazypredict.Supervised import LazyClassifier

In [3]:
# Load dataset
neither_vs_melkeratosis_features_targets = pd.read_csv('../neither_vs_melkeratosis_features_targets.csv')

In [4]:
neither_vs_melkeratosis_features_targets.head()

,image_id,red_median,red_iqr,green_iqr,original_firstorder_10Percentile,original_firstorder_90Percentile,original_firstorder_InterquartileRange,original_firstorder_Range,original_glcm_Autocorrelation,original_gldm_LargeDependenceEmphasis,original_glrlm_LongRunEmphasis,original_glrlm_LongRunLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_ngtdm_Strength,mel_keratosis,neither
0,ISIC_0000000,94,67,65,169.60,182.00,6.00,46.00,7.10,5.23,3.00,0.60,2.63,0.05,0,1
1,ISIC_0000001,116,86,58,112.00,135.00,12.00,72.00,6.02,4.67,1.46,0.30,2.78,0.12,0,1
2,ISIC_0000002,161,26,52,157.00,169.00,6.00,38.00,3.99,5.05,1.50,0.38,1.93,0.94,1,0
3,ISIC_0000003,161,60,57,176.00,191.00,7.00,49.00,3.86,5.16,1.51,0.43,2.21,0.08,0,1
4,ISIC_0000004,192,36,44,26.00,205.00,159.00,210.00,31.01,4.67,1.55,0.27,4.28,2.21,1,0


In [5]:
# Train and test several models

# separate features and target
X = neither_vs_melkeratosis_features_targets.iloc[:,1:-2]
y = neither_vs_melkeratosis_features_targets[['mel_keratosis']]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Preliminary model training and evaluation with LazyClassifier
clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)
print(models)

100%|███████████████████████████████████████████| 29/29 [00:02<00:00,  9.80it/s]

                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
NearestCentroid                    0.67               0.66     0.66      0.68   
PassiveAggressiveClassifier        0.65               0.65     0.65      0.67   
LGBMClassifier                     0.73               0.63     0.63      0.71   
BernoulliNB                        0.70               0.63     0.63      0.70   
BaggingClassifier                  0.74               0.62     0.62      0.72   
RandomForestClassifier             0.74               0.62     0.62      0.72   
GaussianNB                         0.70               0.62     0.62      0.70   
AdaBoostClassifier                 0.74               0.62     0.62      0.71   
XGBClassifier                      0.73               0.62     0.62      0.71   
LogisticRegression                 0.75               0.61     0.61      0.71   
NuSVC                       